# Multiomics BMI Paper — Visualization of Network Interaction Analysis

***by Kengo Watanabe***  

This Jupyter Notebook (with R kernel) visualized the analyte–analyte pairs that were significantly modified by MetBMI (i.e., GLM results) while highlighting the anayte–analyte pairs that were significantly modified by days in program (i.e., GEE results).  

Input files:  
* GEE result summary: 220806_Multiomics-BMI-NatMed1stRevision_LongitudinalNetworkAnalysis-ver2_GEE-interaction.tsv  
* Arivale baseline blood omics: 210104_Biological-BMI-paper_data-cleaning-BMI-omics_baseline-\[metDF/protDF/chemDF\]-without-imputation_final-cohort.tsv  

Output figures and tables:  
* Figure 6a  

Original notebook (memo for my future tracing):  
* dalek:\[JupyterLab HOME\]/220621_Multiomics-BMI-NatMedRevision/220806_Multiomics-BMI-NatMed1stRevision_NetworkAnalysisVisualization.ipynb  

In [1]:
library("tidyverse")
options(repr.plot.width=5, repr.plot.height=5)#Default=7x7

#CRAN
for (package in c("circlize", "RColorBrewer", "colorspace")) {
    #install.packages(package)
    eval(bquote(library(.(package))))
    print(str_c(package, ": ", as.character(packageVersion(package))))
}

#Internal
for (package in c("arivale.data.interface")) {
    eval(bquote(library(.(package))))
    print(str_c(package, ": ", as.character(packageVersion(package))))
}

── Attaching packages ─────────────────────────────────────── tidyverse 1.3.1 ──

✔ ggplot2 3.3.6     ✔ purrr   0.3.4
✔ tibble  3.1.7     ✔ dplyr   1.0.9
✔ tidyr   1.2.0     ✔ stringr 1.4.0
✔ readr   2.1.2     ✔ forcats 0.5.1

── Conflicts ────────────────────────────────────────── tidyverse_conflicts() ──
✖ dplyr::filter() masks stats::filter()
✖ dplyr::lag()    masks stats::lag()

circlize version 0.4.15
CRAN page: https://cran.r-project.org/package=circlize
Github page: https://github.com/jokergoo/circlize
Documentation: https://jokergoo.github.io/circlize_book/book/

If you use it in published research, please cite:
Gu, Z. circlize implements and enhances circular visualization
  in R. Bioinformatics 2014.

This message can be suppressed by:
  suppressPackageStartupMessages(library(circlize))




[1] "circlize: 0.4.15"
[1] "RColorBrewer: 1.1.3"
[1] "colorspace: 2.0.3"
[1] "arivale.data.interface: 0.0.0.1"


## 2. Longitudinal network analysis

> Use all-omics version; but majority is metabolomics.  
> –> Use subcategory for metabolomics.  

### 2-1. Prepare the significant pairs in GLM

In [ ]:
#Import significant analyte pairs in GLM with the subsequent GEE results
fileDir <- "./ExportData/"
ipynbName <- "220806_Multiomics-BMI-NatMed1stRevision_LongitudinalNetworkAnalysis-ver2_"
fileName <- "GEE-interaction.tsv"
pair.tbl <- read_delim(str_c(fileDir, ipynbName, fileName), delim="\t") %>%
    dplyr::select(PairLabel, Variable1, Variable2, Pearson_r,
                  GLM_Bcoef, GLM_AdjPval, GEE_Bcoef, GEE_AdjPval) %>%
    dplyr::filter(GLM_AdjPval < 0.05)
temp <- unique(c(pair.tbl$Variable1, pair.tbl$Variable2))
print(str_c("Significant pairwise alnalyte interactions with MetBMI (FDR < 0.05): ", nrow(pair.tbl)))
print(str_c(" -> nAnalytes: ", length(temp)))

temp.tbl <- pair.tbl %>% dplyr::filter(GEE_AdjPval < 0.05)
temp <- unique(c(temp.tbl$Variable1, temp.tbl$Variable2))
print(str_c("Significant pairwise analyte interactions with days (FDR < 0.05): ", nrow(temp.tbl)))
print(str_c(" -> nAnalytes: ", length(temp)))
pair.tbl <- pair.tbl %>% dplyr::mutate(Highlight=ifelse((GEE_AdjPval < 0.05), 1, 0))

#Convert to -log10 for potential edge attribution
pair.tbl <- pair.tbl %>% dplyr::mutate(GLM_AdjPval_scaled=-log10(GLM_AdjPval))

head(pair.tbl)
tail(pair.tbl)

### 2-2. Prepare the analyte in the significant paris

In [ ]:
#Prepare node table with omics category
node.tbl <- tibble(Analyte=unique(c(pair.tbl$Variable1, pair.tbl$Variable2))) %>%
    dplyr::mutate(Category=rep(0, n()))#Dummy
    
temp <- c("public_client_id", "log_BaseBMI", "Sex", "BaseAge", "PC1", "PC2", "PC3", "PC4", "PC5", "Race")
fileDir <- "../210104_Biological-BMI-paper/ExportData/"
ipynbName <- "210104_Biological-BMI-paper_data-cleaning-BMI-omics_"
fileName <- "baseline-metDF-without-imputation_final-cohort.tsv"
temp.c <- read_delim(str_c(fileDir, ipynbName, fileName), delim="\t") %>%
    dplyr::select(-temp) %>%
    names(.)
print(str_c("Metabolomics: ", length(temp.c)))
node.tbl <- node.tbl %>% dplyr::mutate(Category=ifelse((Analyte %in% temp.c), "Metabolomics", Category))

fileName <- "baseline-protDF-without-imputation_final-cohort.tsv"
temp.c <- read_delim(str_c(fileDir, ipynbName, fileName), delim="\t") %>%
    dplyr::select(-temp) %>%
    names(.)
print(str_c("Proteomics: ", length(temp.c)))
node.tbl <- node.tbl %>% dplyr::mutate(Category=ifelse((Analyte %in% temp.c), "Proteomics", Category))

fileName <- "baseline-chemDF-without-imputation_final-cohort.tsv"
temp.c <- read_delim(str_c(fileDir, ipynbName, fileName), delim="\t") %>%
    dplyr::select(-temp) %>%
    names(.)
print(str_c("Clinical labs: ", length(temp.c)))
node.tbl <- node.tbl %>% dplyr::mutate(Category=ifelse((Analyte %in% temp.c), "Clinical labs", Category))

#Check
node.tbl %>% dplyr::filter(Category==0)
print(str_c("nrow: ", nrow(node.tbl)))
head(node.tbl)

In [ ]:
#Significant analyte in GEE (FDR < 0.05)
temp.tbl <- pair.tbl %>% dplyr::filter(GEE_AdjPval < 0.05)
temp <- unique(c(temp.tbl$Variable1, temp.tbl$Variable2))
node.tbl <- node.tbl %>% dplyr::mutate(Highlight=ifelse((Analyte %in% temp), 1, 0))

#The number of connected edges
temp <- c(pair.tbl$Variable1, pair.tbl$Variable2)
node.tbl <- node.tbl %>%
    dplyr::mutate(nEdges=sapply(Analyte, function(x){length(temp[which(temp==x)])})) %>%
    dplyr::arrange(desc(nEdges))

head(node.tbl)

#Summary
temp.tbl <- node.tbl %>%
    dplyr::group_by(Category) %>%
    dplyr::summarize(Count=n()) %>%
    dplyr::arrange(desc(Count))
temp.tbl
temp.tbl <- node.tbl %>%
    dplyr::filter(Highlight==1) %>%
    dplyr::group_by(Category) %>%
    dplyr::summarize(Count=n()) %>%
    dplyr::arrange(desc(Count))
temp.tbl

### 2-3. Prepare the metabolomics pathway information for subcategory

In [ ]:
#Import the Arivale metabolomics metadata for the significant analytes in GLM
snapshotPath <- "/shared-data/snapshots/arivale_snapshot_ISB_2019-05-19_1330"
node.tbl <- get_snapshot("metabolomics_metadata", path=snapshotPath) %>%
    as_tibble() %>%
    dplyr::select(BIOCHEMICAL_NAME, SUPER_PATHWAY, SUB_PATHWAY) %>%
    dplyr::rename(Analyte=BIOCHEMICAL_NAME) %>%
    dplyr::left_join(node.tbl, ., by="Analyte")

#Fill empty as string "NA"
node.tbl <- node.tbl %>%
    dplyr::mutate(SUPER_PATHWAY=ifelse(SUPER_PATHWAY=="", "N.A.", SUPER_PATHWAY),
                  SUB_PATHWAY=ifelse(SUB_PATHWAY=="", "N.A.", SUB_PATHWAY))

head(node.tbl)
tail(node.tbl)
summary(node.tbl)
print(str_c("Unique SUPER_PATHWAY: ", length(unique(node.tbl$SUPER_PATHWAY))))
temp.tbl <- node.tbl %>%
    dplyr::group_by(SUPER_PATHWAY) %>%
    dplyr::summarize(Count=n()) %>%
    dplyr::arrange(desc(Count))
temp.tbl
print(str_c("Unique SUB_PATHWAY: ", length(unique(node.tbl$SUB_PATHWAY))))
temp.tbl <- node.tbl %>%
    dplyr::group_by(SUPER_PATHWAY, SUB_PATHWAY) %>%
    dplyr::summarize(Count=n()) %>%
    dplyr::ungroup() %>%
    dplyr::arrange(desc(Count))
temp <- unique(temp.tbl$SUPER_PATHWAY)
for (pathway in temp){
    print(pathway)
    temp.tbl %>% dplyr::filter(SUPER_PATHWAY==pathway) %>% print()
}

> • Use SUB_PATHWAY name if the category has more than 4 metabolites; otherwise, combine to "Others (SUPER_PATHWAY)".  
> • "Carbohydrate", "Nucleotide", "Cofactors and Vitamins", "Energy", "Peptide", and "Xenobiotics" are combined to "Others (Carbohydrate, Nucleotide, etc.)".  
> • Proteomics and Clinical labs are filled with label manually.  

In [ ]:
#Combine small SUPER_PATHWAYs
temp.c <- c("Carbohydrate", "Nucleotide", "Cofactors and Vitamins", "Energy", "Peptide", "Xenobiotics")
node.tbl <- node.tbl %>%
    dplyr::mutate(SUPER_PATHWAY=ifelse((SUPER_PATHWAY %in% temp.c),
                                       "Others (Carbohydrate, Nucleotide, etc.)", SUPER_PATHWAY))

#Clean subcategory for the SUB_PATHWAY with <4 metabolites
temp.tbl <- node.tbl %>%
    dplyr::group_by(SUPER_PATHWAY, SUB_PATHWAY) %>%
    dplyr::summarize(Count=n()) %>%
    dplyr::ungroup() %>%
    dplyr::arrange(desc(Count)) %>%
    dplyr::filter(Count>=4)#Including proteomics and clinical labs as NA; and empty "N.A."
node.tbl <- node.tbl %>%
    dplyr::mutate(Subcategory=ifelse((SUB_PATHWAY %in% unique(temp.tbl$SUB_PATHWAY)),
                                     SUB_PATHWAY, str_c("Others (", SUPER_PATHWAY, ")"))) %>%
    #dplyr::mutate(Subcategory=ifelse((SUPER_PATHWAY=="N.A."), "N.A.", Subcategory)) %>%
    dplyr::mutate(Subcategory=ifelse((SUPER_PATHWAY=="Others (Carbohydrate, Nucleotide, etc.)"),
                                     "Others (Carbohydrate, Nucleotide, etc.)", Subcategory)) %>%
    dplyr::mutate(Subcategory=ifelse((Category=="Proteomics"), "Protein", Subcategory)) %>%
    dplyr::mutate(Subcategory=ifelse((Category=="Clinical labs"), "Clinical Lab Test", Subcategory))

#Convert inconsistent capitalization of label
node.tbl <- node.tbl %>%
    dplyr::mutate(Subcategory=ifelse(Subcategory=="Urea cycle; Arginine and Proline Metabolism",
                                     "Urea Cycle; Arginine and Proline Metabolism", Subcategory))

print(str_c("nrows: ", nrow(node.tbl)))
head(node.tbl)
tail(node.tbl)
print(str_c("Unique subcategory: ", length(unique(node.tbl$Subcategory))))
temp.tbl <- node.tbl %>%
    dplyr::group_by(Category, Subcategory) %>%
    dplyr::summarize(Count=n()) %>%
    dplyr::ungroup() %>%
    dplyr::arrange(desc(Count))
temp.tbl

### 2-4. Prepare the sector table for visualization

> The combined small SUPER_PATHWAYs are already updated in the SUPER_PATHWAY column.  

In [ ]:
sector.tbl <- node.tbl %>%
    dplyr::select(Category, SUPER_PATHWAY, Subcategory) %>%
    dplyr::distinct()

#Numbering category order for visualization
sector.tbl <- node.tbl %>%
    dplyr::group_by(Category) %>%
    dplyr::summarize(DummyCount=n()) %>%
    dplyr::arrange(desc(DummyCount)) %>%
    dplyr::mutate(CatNum=1:n()) %>%
    dplyr::select(-DummyCount) %>%
    dplyr::left_join(sector.tbl, ., by="Category")

#Numbering subcategory order for visualization ("N.A." and "Others (xxx)" should be the last)
sector.tbl <- node.tbl %>%
    dplyr::group_by(SUPER_PATHWAY) %>%
    dplyr::summarize(DummyCount=n()) %>%
    dplyr::mutate(DummyCount=ifelse((SUPER_PATHWAY=="N.A."|is.na(SUPER_PATHWAY)), NA, DummyCount)) %>%
    dplyr::arrange(desc(DummyCount)) %>%
    dplyr::mutate(DummyOrder=1:n()) %>%
    dplyr::select(-DummyCount) %>%
    dplyr::left_join(sector.tbl, ., by="SUPER_PATHWAY")
sector.tbl <- node.tbl %>%
    dplyr::group_by(Subcategory) %>%
    dplyr::summarize(Count=n()) %>%
    dplyr::mutate(DummyCount=ifelse(str_detect(Subcategory, pattern="Others "), NA, Count)) %>%
    dplyr::left_join(sector.tbl, ., by="Subcategory") %>%
    dplyr::arrange(CatNum, DummyOrder, desc(DummyCount)) %>%
    dplyr::group_by(Category) %>%
    dplyr::mutate(SubcatNum=1:n()) %>%
    dplyr::ungroup() %>%
    dplyr::mutate(SectorNum=1:n()) %>%
    dplyr::select(-DummyOrder, -DummyCount)
sector.tbl

In [ ]:
360/nrow(node.tbl)

> –> When setting the gap degree b/w categories (dummy sectors in circos plot) to 5, x coordinate between subcategories (true sectors) should be less than 5 but more than 2.75.  
> –> In this case, add 0.3 to Xcood as a gap between subcategories, while assuming the gap degree 3–4.  

In [ ]:
#Calculate the first/last x coordinate of subcategory on each category (dummy sector in circos plot)
temp <- c()
gap <- 0.3
for (row_i in 1:nrow(sector.tbl)) {
    subcat_i <- sector.tbl$SubcatNum[row_i]
    if (subcat_i==1) {
        temp <- c(temp, 1)
    } else {
        temp.tbl <- sector.tbl %>%
            dplyr::filter(CatNum==sector.tbl$CatNum[row_i])
        total_before <- sum(temp.tbl$Count[1:(subcat_i-1)])
        temp <- c(temp, (total_before + gap*(subcat_i-1) + 1))
    }
}
sector.tbl$SubcatXmin <- temp
sector.tbl$SubcatXmax <- sector.tbl$SubcatXmin + sector.tbl$Count - 1

#Calculate the first/last x corrdinate of category (dummy sector in circos plot)
sector.tbl <- sector.tbl %>%
    dplyr::group_by(Category) %>%
    dplyr::summarize(CatXmin=min(SubcatXmin),
                     CatXmax=max(SubcatXmax)) %>%
    dplyr::left_join(sector.tbl, ., by="Category")

sector.tbl

### 2-5. Calculate x coordinate of analyte

In [ ]:
#Calculate the x coordinate of node on each category (dummy sector in circos plot)
node.tbl <- sector.tbl %>%
    dplyr::select(Subcategory, SectorNum, SubcatXmin, SubcatXmax) %>%
    dplyr::left_join(node.tbl, ., by="Subcategory") %>%
    dplyr::arrange(SectorNum, desc(nEdges), Analyte) %>%
    dplyr::group_by(Subcategory) %>%
    dplyr::mutate(DummyXcoord=1:n()) %>%
    dplyr::ungroup() %>%
    dplyr::mutate(Xcoord=SubcatXmin+DummyXcoord-1) %>%
    dplyr::select(-DummyXcoord)
head(node.tbl)
print("Check margin")
node.tbl[16:26,]
node.tbl[80:85,]

### 2-6. Calculate label offset amount

> –> Skip this step because all labels are used this case.  

### 2-7. Circos plot

In [ ]:
#Check
head(sector.tbl)
head(node.tbl)
head(pair.tbl)

In [ ]:
options(repr.plot.width=10, repr.plot.height=10)#Default=7x7
circos.clear()

#Generate xlim table
sector.tbl.cat <- sector.tbl %>%
    dplyr::select(Category, CatNum, CatXmin, CatXmax) %>%
    dplyr::distinct()

#Initialize
clockwise <- TRUE
circos.par(start.degree=88, gap.degree=4, track.margin=c(0, 0),
           cell.padding=c(0, 0, 0, 0), unit.circle.segments=nrow(node.tbl),
           canvas.xlim=c(-2.3, 2.7), canvas.ylim=c(-2.35, 2.65),#Change based on the label length
           clock.wise=clockwise, xaxis.clock.wise=clockwise)
circos.initialize(sectors=sector.tbl.cat$CatNum, xlim=sector.tbl.cat[, c("CatXmin", "CatXmax")],
                  sector.width=sector.tbl.cat$CatXmax)

#Prepare color palette with dummy sector order
color_palette <- brewer.pal(nrow(sector.tbl.cat), "Set1")
color_palette <- darken(color_palette, amount=0.05)

#1st dummy track
circos.track(ylim=c(0, 1), track.height=0.02, bg.col=NA, bg.border=NA)
for (row_i in 1:nrow(node.tbl)) {
    #Add node label
    category <- node.tbl$Category[row_i]
    sector <- sector.tbl.cat$CatNum[sector.tbl.cat$Category==category]
    xcoord <- node.tbl$Xcoord[row_i]
    label <- node.tbl$Analyte[row_i]
    if (category=="Metabolomics") {
        text_color <- color_palette[2]#blue
    } else if (category=="Proteomics") {
        text_color <- color_palette[1]#red
    } else if (category=="Clinical labs") {
        text_color <- color_palette[3]#green
    }        
    if (node.tbl$Highlight[row_i]==1) {
        text_cex <- 0.75
        text_style <- 2#bold font
    } else if (node.tbl$Highlight[row_i]==0) {
        text_cex <- 0.4
        text_style <- 1
    }
    circos.text(x=xcoord, y=1, labels=label, sector.index=sector, track.index=1,
                facing="clockwise", niceFacing=TRUE, adj=c(0, 0.5),
                cex=text_cex, col=text_color, font=text_style)
    #Add axis ticks manually (circos.axis is not suitable in this case)
    ##<- Ticks poisition can be adjusted, but the baseline is remained.
    line_height <- 0.75
    line_color <- "black"
    line_width <- 1
    circos.lines(x=rep(xcoord, 2), y=c(0, line_height),
                 sector.index=sector, track.index=1, col=line_color, lwd=line_width)
}

#2nd dummy track
circos.track(ylim=c(0, 1), track.height=0.1, bg.col=NA, bg.border=NA)
for (row_i in 1:nrow(sector.tbl)) {
    sector <- sector.tbl$CatNum[row_i]
    xstart <- sector.tbl$SubcatXmin[row_i]
    xend <- sector.tbl$SubcatXmax[row_i]
    theta_start <- circlize(x=xstart, y=0, sector.index=sector, track.index=2)[1, 1]
    theta_end <- circlize(x=xend, y=0, sector.index=sector, track.index=2)[1, 1]
    #Add subcategory (true) sectors
    if (sector.tbl$Category[row_i]=="Metabolomics") {
        fill_color <- color_palette[2]#blue
    } else if (sector.tbl$Category[row_i]=="Proteomics") {
        fill_color <- color_palette[1]#red
    } else if (sector.tbl$Category[row_i]=="Clinical labs") {
        fill_color <- color_palette[3]#green
    }
    draw.sector(start.degree=theta_start, end.degree=theta_end, clock.wise=clockwise,
                rou1=get.cell.meta.data("cell.top.radius", track.index=2),
                rou2=get.cell.meta.data("cell.bottom.radius", track.index=2),
                col=fill_color, border = "black", lwd=2)
    #Add subcategory number
    xcenter <- (xstart + xend) / 2
    label <- str_c("#", as.character(sector.tbl$SectorNum[row_i]))
    circos.text(x=xcenter, y=0.5, labels=label, sector.index=sector, track.index=2,
                facing="bending.inside", niceFacing=TRUE, adj=c(0.5, 0.5),
                cex=0.8, col="white", font=2)#2 = bold font
}

#Add links
for (row_i in 1:nrow(pair.tbl)) {
    var1 <- pair.tbl$Variable1[row_i]
    var2 <- pair.tbl$Variable2[row_i]
    var1_category <- node.tbl$Category[node.tbl$Analyte==var1]
    var2_category <- node.tbl$Category[node.tbl$Analyte==var2]
    var1_sector <- sector.tbl.cat$CatNum[sector.tbl.cat$Category==var1_category]
    var2_sector <- sector.tbl.cat$CatNum[sector.tbl.cat$Category==var2_category]
    var1_xcoord <- node.tbl$Xcoord[node.tbl$Analyte==var1]
    var2_xcoord <- node.tbl$Xcoord[node.tbl$Analyte==var2]
    if (pair.tbl$GLM_Bcoef[row_i]>0) {
        if (pair.tbl$Highlight[row_i]==1) {
            line_color=rgb(1, 0, 0, alpha=0.6)
            line_width=1
        } else if (pair.tbl$Highlight[row_i]==0) {
            line_color=rgb(1, 0, 0, alpha=0.2)
            line_width=0.3
        }
    } else if (pair.tbl$GLM_Bcoef[row_i]<0) {
        if (pair.tbl$Highlight[row_i]==1) {
            line_color=rgb(0, 0, 1, alpha=0.6)
            line_width=1
        } else if (pair.tbl$Highlight[row_i]==0) {
            line_color=rgb(0, 0, 1, alpha=0.2)
            line_width=0.3
        }
    }
    circos.link(sector.index1=var1_sector, point1=var1_xcoord,
                sector.index2=var2_sector, point2=var2_xcoord,
                h.ratio=0.6, w=4, col=line_color, lwd=line_width)
}

circos.clear()
options(repr.plot.width=5, repr.plot.height=5)#Default=7x7

> Because circlize package uses base graphycs, use graphycs device.  

In [ ]:
options(repr.plot.width=10, repr.plot.height=10)#Default=7x7
circos.clear()

#To save (circlize uses base graphycs)
fileDir = "./ExportFigures/"
ipynbName <- "220806_Multiomics-BMI-NatMed1stRevision_NetworkAnalysisVisualization_"
fileName <- "longitudinal.pdf"
pdf(str_c(fileDir, ipynbName, fileName), 10, 10)

#Generate xlim table
sector.tbl.cat <- sector.tbl %>%
    dplyr::select(Category, CatNum, CatXmin, CatXmax) %>%
    dplyr::distinct()

#Initialize
clockwise <- TRUE
circos.par(start.degree=88, gap.degree=4, track.margin=c(0, 0),
           cell.padding=c(0, 0, 0, 0), unit.circle.segments=nrow(node.tbl),
           canvas.xlim=c(-2.3, 2.7), canvas.ylim=c(-2.35, 2.65),#Change based on the label length
           clock.wise=clockwise, xaxis.clock.wise=clockwise)
circos.initialize(sectors=sector.tbl.cat$CatNum, xlim=sector.tbl.cat[, c("CatXmin", "CatXmax")],
                  sector.width=sector.tbl.cat$CatXmax)

#Prepare color palette with dummy sector order
color_palette <- brewer.pal(nrow(sector.tbl.cat), "Set1")
color_palette <- darken(color_palette, amount=0.05)

#1st dummy track
circos.track(ylim=c(0, 1), track.height=0.02, bg.col=NA, bg.border=NA)
for (row_i in 1:nrow(node.tbl)) {
    #Add node label
    category <- node.tbl$Category[row_i]
    sector <- sector.tbl.cat$CatNum[sector.tbl.cat$Category==category]
    xcoord <- node.tbl$Xcoord[row_i]
    label <- node.tbl$Analyte[row_i]
    if (category=="Metabolomics") {
        text_color <- color_palette[2]#blue
    } else if (category=="Proteomics") {
        text_color <- color_palette[1]#red
    } else if (category=="Clinical labs") {
        text_color <- color_palette[3]#green
    }        
    if (node.tbl$Highlight[row_i]==1) {
        text_cex <- 0.75
        text_style <- 2#bold font
    } else if (node.tbl$Highlight[row_i]==0) {
        text_cex <- 0.4
        text_style <- 1
    }
    circos.text(x=xcoord, y=1, labels=label, sector.index=sector, track.index=1,
                facing="clockwise", niceFacing=TRUE, adj=c(0, 0.5),
                cex=text_cex, col=text_color, font=text_style)
    #Add axis ticks manually (circos.axis is not suitable in this case)
    ##<- Ticks poisition can be adjusted, but the baseline is remained.
    line_height <- 0.75
    line_color <- "black"
    line_width <- 1
    circos.lines(x=rep(xcoord, 2), y=c(0, line_height),
                 sector.index=sector, track.index=1, col=line_color, lwd=line_width)
}

#2nd dummy track
circos.track(ylim=c(0, 1), track.height=0.1, bg.col=NA, bg.border=NA)
for (row_i in 1:nrow(sector.tbl)) {
    sector <- sector.tbl$CatNum[row_i]
    xstart <- sector.tbl$SubcatXmin[row_i]
    xend <- sector.tbl$SubcatXmax[row_i]
    theta_start <- circlize(x=xstart, y=0, sector.index=sector, track.index=2)[1, 1]
    theta_end <- circlize(x=xend, y=0, sector.index=sector, track.index=2)[1, 1]
    #Add subcategory (true) sectors
    if (sector.tbl$Category[row_i]=="Metabolomics") {
        fill_color <- color_palette[2]#blue
    } else if (sector.tbl$Category[row_i]=="Proteomics") {
        fill_color <- color_palette[1]#red
    } else if (sector.tbl$Category[row_i]=="Clinical labs") {
        fill_color <- color_palette[3]#green
    }
    draw.sector(start.degree=theta_start, end.degree=theta_end, clock.wise=clockwise,
                rou1=get.cell.meta.data("cell.top.radius", track.index=2),
                rou2=get.cell.meta.data("cell.bottom.radius", track.index=2),
                col=fill_color, border = "black", lwd=2)
    #Add subcategory number
    xcenter <- (xstart + xend) / 2
    label <- str_c("#", as.character(sector.tbl$SectorNum[row_i]))
    circos.text(x=xcenter, y=0.5, labels=label, sector.index=sector, track.index=2,
                facing="bending.inside", niceFacing=TRUE, adj=c(0.5, 0.5),
                cex=0.8, col="white", font=2)#2 = bold font
}

#Add links
for (row_i in 1:nrow(pair.tbl)) {
    var1 <- pair.tbl$Variable1[row_i]
    var2 <- pair.tbl$Variable2[row_i]
    var1_category <- node.tbl$Category[node.tbl$Analyte==var1]
    var2_category <- node.tbl$Category[node.tbl$Analyte==var2]
    var1_sector <- sector.tbl.cat$CatNum[sector.tbl.cat$Category==var1_category]
    var2_sector <- sector.tbl.cat$CatNum[sector.tbl.cat$Category==var2_category]
    var1_xcoord <- node.tbl$Xcoord[node.tbl$Analyte==var1]
    var2_xcoord <- node.tbl$Xcoord[node.tbl$Analyte==var2]
    if (pair.tbl$GLM_Bcoef[row_i]>0) {
        if (pair.tbl$Highlight[row_i]==1) {
            line_color=rgb(1, 0, 0, alpha=0.6)
            line_width=1
        } else if (pair.tbl$Highlight[row_i]==0) {
            line_color=rgb(1, 0, 0, alpha=0.2)
            line_width=0.3
        }
    } else if (pair.tbl$GLM_Bcoef[row_i]<0) {
        if (pair.tbl$Highlight[row_i]==1) {
            line_color=rgb(0, 0, 1, alpha=0.6)
            line_width=1
        } else if (pair.tbl$Highlight[row_i]==0) {
            line_color=rgb(0, 0, 1, alpha=0.2)
            line_width=0.3
        }
    }
    circos.link(sector.index1=var1_sector, point1=var1_xcoord,
                sector.index2=var2_sector, point2=var2_xcoord,
                h.ratio=0.6, w=4, col=line_color, lwd=line_width)
}

#Close
dev.off()

circos.clear()
options(repr.plot.width=5, repr.plot.height=5)#Default=7x7

> –> The exported figure is surely .pdf file.  
> –> Furthermore, although different font was used on the output cell in Jupyter notebook, the Arial font was correctly used in the exported file!!  

# — Session information —

In [14]:
sessionInfo()

R version 4.1.1 (2021-08-10)
Platform: x86_64-conda-linux-gnu (64-bit)
Running under: Ubuntu 20.04.3 LTS

Matrix products: default
BLAS/LAPACK: /opt/conda/envs/arivale-r/lib/libopenblasp-r0.3.18.so

locale:
 [1] LC_CTYPE=en_US.UTF-8       LC_NUMERIC=C              
 [3] LC_TIME=en_US.UTF-8        LC_COLLATE=en_US.UTF-8    
 [5] LC_MONETARY=en_US.UTF-8    LC_MESSAGES=en_US.UTF-8   
 [7] LC_PAPER=en_US.UTF-8       LC_NAME=C                 
 [9] LC_ADDRESS=C               LC_TELEPHONE=C            
[11] LC_MEASUREMENT=en_US.UTF-8 LC_IDENTIFICATION=C       

attached base packages:
[1] stats     graphics  grDevices utils     datasets  methods   base     

other attached packages:
 [1] arivale.data.interface_0.0.0.1 colorspace_2.0-3              
 [3] RColorBrewer_1.1-3             circlize_0.4.15               
 [5] forcats_0.5.1                  stringr_1.4.0                 
 [7] dplyr_1.0.9                    purrr_0.3.4                   
 [9] readr_2.1.2                    tidyr_1.2.